# Card Game with Augmented Reality

In [989]:
import cv2
import numpy as np
import os
import math

In [990]:
database_folder_path = "./resized"
names_database_foldername = "names"
suits_database_foldername = "suits"
names_database_path = f"./{names_database_foldername}"
suits_database_path = f"./{suits_database_foldername}"
img_type = "png"
frame_test_path = "./frames_test/frame_2.jpg"

card_simple_rect = (30, 40, 130, 210)
card_simple_factor = 2

In [991]:
def get_number_black_pixels(img):
    return np.sum(img == 0)

In [992]:
def get_center_point(points):
    n_points, _ = points.shape
    acc_x = 0
    acc_y = 0
    for point in points:
        acc_x += point[0]
        acc_y += point[1]

    return acc_x / n_points, acc_y / n_points

def get_vector_norm(v):
    return math.sqrt(v[0]**2 + v[1]**2)

def get_vectors_angle(v1, v2):
    dot_product = v1[0]*v2[0] + v1[1]*v2[1]
    v1_norm = get_vector_norm(v1)
    v2_norm = get_vector_norm(v2)
    return math.acos(dot_product/(v1_norm*v2_norm))

def sort_points(points):
    final = []
    aux = []
    ref_vector = (1, 0)
    center = get_center_point(points)
    for point in points:
        v = (point[0]-center[0], point[1]-center[1])
        angle = get_vectors_angle(v, ref_vector)
        if (point[1]-center[1] < 0):
            angle = 2*math.pi - angle
        aux.append((angle, point))

    aux.sort()
    for _, point in aux:
        final.append(list(point))

    return np.array([np.array(i) for i in final])

In [993]:
class Card:
    def __init__(self, name, suit, img) -> None:
        self.__name = name
        self.__suit = suit
        self.__img = img
        self.__name_img = None
        self.__suit_img = None
        self.update_name_suit_img()

    def get_name(self):
        return self.__name

    def get_suit(self):
        return self.__suit

    def get_img(self):
        return self.__img

    def update_name_suit_img(self):
        name_img_path = f"./{names_database_foldername}/{self.__name}.png"
        suit_img_path = f"./{suits_database_foldername}/{self.__suit}.png"
        self.__name_img = cv2.imread(name_img_path)
        self.__suit_img = cv2.imread(name_img_path) 

In [994]:
def cnt1_is_inside_cnt2(cnt1, cnt2):
    cnt1_x1, cnt1_y1, cnt1_w, cnt1_h = cv2.boundingRect(cnt1)
    cnt1_x2 = cnt1_x1 + cnt1_w
    cnt1_y2 = cnt1_y1 + cnt1_h
    cnt2_x1, cnt2_y1, cnt2_w, cnt2_h = cv2.boundingRect(cnt2)
    cnt2_x2 = cnt2_x1 + cnt2_w
    cnt2_y2 = cnt2_y1 + cnt2_h

    return (cnt1_x1 >= cnt2_x1) and (cnt1_y1 >= cnt2_y1) and (cnt1_x2 <= cnt2_x2) and (cnt1_y2 <= cnt2_y2)

In [995]:
def get_card_simple_area():
    x1, y1, x2, y2 = card_simple_rect
    return (x2 - x1) * (y2 - y1) * card_simple_factor ** 2

In [996]:
def extract_card_simple(card_img):
    x1, y1, x2, y2 = card_simple_rect
    card_simple = card_img[y1:y2, x1:x2]
    card_simple = cv2.resize(card_simple, (0,0), fx=card_simple_factor, fy=card_simple_factor) 
    return card_simple
'''
def extract_card_name_suit_images(card_img):
    x1, y1, x2, y2 = card_simple_rect
    h = y2 - y1
    card_img_simple = card_img[y1:y2, x1:x2]
    name_img = card_img_simple[:h//2,:]
    suit_img = card_img_simple[h//2+1:,:]
    return name_img, suit_img
'''


'\ndef extract_card_name_suit_images(card_img):\n    x1, y1, x2, y2 = card_simple_rect\n    h = y2 - y1\n    card_img_simple = card_img[y1:y2, x1:x2]\n    name_img = card_img_simple[:h//2,:]\n    suit_img = card_img_simple[h//2+1:,:]\n    return name_img, suit_img\n'

In [997]:
def create_binary(frame):
    # convert to gray
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # apply blur
    frame = cv2.GaussianBlur(frame, (3,3), 0)

    # convert to binary
    frame = cv2.threshold(frame, 210, 255, cv2.THRESH_BINARY)[1]

    return frame

In [998]:
def extract_card_name_suit(filename):
    #suit_legend = {"C": "Clubs", "D": "Diamonds", "H": "Hearts", "S": "Spades"}
    name = filename.split(".")[0]
    name_split = name.split("_")
    return name_split[0], name_split[1]

def extract_card_name_or_suit(filename):
    return filename.split(".")[0]

def read_all_cards():
    cards = set()

    filenames = os.listdir(database_folder_path)
    for filename in filenames:
        frame = cv2.imread(f"{database_folder_path}/{filename}")
        binary_frame = create_binary(frame)
        name, suit = extract_card_name_suit(filename)
        cards.add(Card(name, suit, binary_frame))

    return cards

def read_all_names():
    names = {}

    filenames = os.listdir(names_database_foldername)
    for filename in filenames:
        name_img = cv2.imread(f"{names_database_path}/{filename}")
        binary_frame = create_binary(name_img)
        names[filename] = binary_frame

    return names

def read_all_suits():
    suits = {}

    filenames = os.listdir(suits_database_foldername)
    for filename in filenames:
        suit_img = cv2.imread(f"{suits_database_path}/{filename}")
        binary_frame = create_binary(suit_img)
        suits[filename] = binary_frame

    return suits


In [999]:
def find_contourns(binary, mode):
    cnts = cv2.findContours(binary, mode, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    #for cnt in cnts:
    #    approx = cv2.convexHull(cnt)
    #    approx = cv2.approxPolyDP(approx,0.01*cv2.arcLength(cnt,True),True)

    return cnts

In [1000]:
def filter_contourns(cnts):
    candidates = []
    candidates_to_dups = []
    simple_x1, simple_y1, simple_x2, simple_y2 = card_simple_rect
    simple_w = (simple_x2 - simple_x1) * card_simple_factor
    simple_h = (simple_y2 - simple_y1) * card_simple_factor

    for cnt in cnts:
        cnt_x, cnt_y, cnt_w, cnt_h = cv2.boundingRect(cnt)
        area_cnt = cnt_w * cnt_h

        # Area ration verification
        if area_cnt / get_card_simple_area() > 0.7:
            continue

        # Position verification
        if cnt_x / simple_w > 0.5:
            continue

        candidates_to_dups.append(cnt)

    # Duplicates
    for i in range(len(candidates_to_dups)):
        is_inside = False
        for j in range(len(candidates_to_dups)):
            if i != j:
                cnt1 = candidates_to_dups[i]
                cnt2 = candidates_to_dups[j]
                if cnt1_is_inside_cnt2(cnt1, cnt2):
                    is_inside = True
                    break
        if not is_inside:
            candidates.append(cnt1)

    return candidates

In [1001]:
def find_corners_contourn(cnt, binary_frame):
    # cnt[:, :, 0] -> all x values
    # cnt[:, :, 1] -> all y values
    # cnt[:, :, 0].argmin() -> index min x value
    # cnt[cnt[:, :, 0].argmin()][0] -> pair min x value
    #left_point = tuple(cnt[cnt[:, :, 0].argmin()][0])
    #right_point = tuple(cnt[cnt[:, :, 0].argmax()][0])
    #top_point = tuple(cnt[cnt[:, :, 1].argmin()][0])
    #botton_point = tuple(cnt[cnt[:, :, 1].argmax()][0])
    #print(cnt.shape)
    #print(f"{left_point},{right_point}, {top_point}, {botton_point}")
    #print(cnt)
    #print(cnt[cnt[:, :, 0].argmin()][0])
    #return left_point, right_point, top_point, botton_point


    # https://stackoverflow.com/questions/50984205/how-to-find-corners-points-of-a-shape-in-an-image-in-opencv
    rect = cv2.minAreaRect(cnt)

    binary_frame = np.float32(binary_frame)
    mask = np.zeros(binary_frame.shape, dtype="uint8")
    cv2.fillPoly(mask, [cnt], (255,255,255))
    dst = cv2.cornerHarris(mask,60,7,0.04)
    ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(binary_frame,np.float32(centroids),(5,5),(-1,-1),criteria)
    corners = corners[1:]
    #print("Pau")
    #print(corners)
    #corners = imutils.perspective.order_points(corners)
    corners = sort_points(corners)

    return corners



In [1002]:
def extract_card_name_suit_images_from_contourns(simple_img, cnts):
    name_cnt = cnts[1]
    suit_cnt = cnts[0]

    name_cnt_x, name_cnt_y, name_cnt_w, name_cnt_h = cv2.boundingRect(name_cnt)
    suit_cnt_x, suit_cnt_y, suit_cnt_w, suit_cnt_h = cv2.boundingRect(suit_cnt)

    name_img = simple_img[name_cnt_y:name_cnt_y+name_cnt_h, name_cnt_x:name_cnt_x+name_cnt_w]
    suit_img = simple_img[suit_cnt_y:suit_cnt_y+suit_cnt_h, suit_cnt_x:suit_cnt_x+suit_cnt_w]

    return name_img, suit_img

In [1003]:
def extract_card_name_suit_images(img):
    simple_img = extract_card_simple(img)
    binary_simple_img = create_binary(simple_img)
    simple_img_cnts = find_contourns(binary_simple_img)
    simple_img_cnts = filter_contourns(simple_img_cnts)
    name_img, suit_img = extract_card_name_suit_images_from_contourns(binary_simple_img, simple_img_cnts)
    return name_img, suit_img

In [1004]:
def apply_homography(corners, frame, correction=False):
    database_width = 500
    database_height = 726
    if not correction:
        database_points = np.array([[0, database_height], [0, 0], [database_width, 0], [database_width, database_height]])
    else:
        database_points = np.array([[database_width, database_height], [0, database_height], [0, 0], [database_width, 0]])
    h, status = cv2.findHomography(corners, database_points)
    warped_frame = cv2.warpPerspective(frame, h, (database_width, database_height))
    return warped_frame



In [1005]:
def extract_frame_cards(frame):
    frame_height, frame_width, _ = frame.shape
    card_size_threshold = 0.1
    frame_cards = []
    
    binary_frame = create_binary(frame)
    cnts = find_contourns(binary_frame, cv2.RETR_EXTERNAL)

    #corners = find_corners(binary_frame)
    #corners = cv2.dilate(corners, None)
    #frame[corners>0.01*corners.max()]=[0,0,255]
    
    #for i in corners:
    #    x,y = i.ravel()
    #    cv2.circle(frame,(int(x),int(y)),3,(255, 0, 0),-1)

    #left_point, right_point, top_point, botton_point = find_corners_contourn(cnts[0])
    #cv2.circle(frame, left_point, 8, (0, 0, 255), -1)
    #cv2.circle(frame, right_point, 8, (0, 255, 0), -1)
    #cv2.circle(frame, top_point, 8, (255, 0, 0), -1)
    #cv2.circle(frame, botton_point, 8, (255, 255, 0), -1)

    n_valid_cnts = 0
    i = 0
    for cnt in cnts:
        cnt_x, cnt_y, cnt_width, cnt_height = cv2.boundingRect(cnt)
        if (cnt_width < frame_width * card_size_threshold) or (cnt_height < frame_height * card_size_threshold):
            continue
        n_valid_cnts += 1
        #cv2.drawContours(frame,[cnt],-1,(255,0,0),3)
        frame_card = binary_frame[cnt_y:cnt_y+cnt_height, cnt_x:cnt_x+cnt_width]

        corners = find_corners_contourn(cnt, binary_frame)
        #print(corners.shape)
        for corner in corners:
            #print(tuple(corner))
            cv2.circle(frame, (int(corner[0]), int(corner[1])), 8, (0, 0, 255), -1)

        warped_frame_card = apply_homography(corners, binary_frame)
        simple_frame = extract_card_simple(warped_frame_card)
        #binary_simple_frame = create_binary(simple_frame)
        simple_frame_cnts = find_contourns(simple_frame, cv2.RETR_TREE)
        simple_frame_cnts = filter_contourns(simple_frame_cnts)

        if len(simple_frame_cnts) < 2:
            warped_frame_card = apply_homography(corners, binary_frame, correction=True)
            simple_frame = extract_card_simple(warped_frame_card)
            #binary_simple_frame = create_binary(simple_frame)
            simple_frame_cnts = find_contourns(simple_frame, cv2.RETR_TREE)
            simple_frame_cnts = filter_contourns(simple_frame_cnts)

        #color_simple_frame = cv2.cvtColor(simple_frame, cv2.COLOR_GRAY2RGB)
        
        #print(len(simple_frame_cnts))
        #cv2.drawContours(color_simple_frame,simple_frame_cnts,-1,(255,0,0),3)
        #cv2.imshow(f"Ola {i}", color_simple_frame)

        name_img, suit_img = extract_card_name_suit_images_from_contourns(simple_frame, simple_frame_cnts)

        #warped_frame_card_simple = extract_card_simple(warped_frame_card)
        #card_name_img, card_suit_img = extract_card_name_suit_images(warped_frame_card)

        #n_black_pixels = get_number_black_pixels(card_suit_img)
        #x1, y1, x2, y2 = card_simple_rect
        #n_pixels = (x2 - x1) * (y2 - y1)

        #if (n_black_pixels / n_pixels < 0.1):
        #    warped_frame_card = apply_homography(corners, binary_frame, correction=True)
            #warped_frame_card_simple = extract_card_simple(warped_frame_card)
        #    card_name_img, card_suit_img = extract_card_name_suit_images(warped_frame_card)

        #n_black_pixels = get_number_black_pixels(card_suit_img)

        #cv2.imshow(f"{i}", warped_frame_card)
        #cv2.imshow(f"Name {i}", name_img)
        #cv2.imshow(f"Suit {i}", suit_img)

        #corners = find_corners(frame_card)
        #corners = cv2.dilate(corners, None)
        #frame_card_color = frame_card.copy()
        #frame_card_color = cv2.cvtColor(frame_card_color, cv2.COLOR_GRAY2RGB)
        #frame_card_color[corners>0.01*corners.max()]=[0,0,255]
        #cv2.imshow(f"frame card color {i}", frame_card_color)
        #cv2.imshow(f"{i}", warped_frame_card)
        i += 1

        '''
        cnt_rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(cnt_rect)
        box = np.int0(box)

        # get width and height of the detected rectangle
        cnt_width = int(cnt_rect[1][0])
        cnt_height = int(cnt_rect[1][1])

        src_pts = box.astype("float32")
        # coordinate of the points in box points after the rectangle has been
        # straightened
        dst_pts = np.array([[0, cnt_height-1],
                            [0, 0],
                            [cnt_width-1, 0],
                            [cnt_width-1, cnt_height-1]], dtype="float32")

        # the perspective transformation matrix
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)

        # directly warp the rotated rectangle to get the straightened rectangle
        frame_card = cv2.warpPerspective(frame, M, (cnt_width, cnt_height))
        '''

        #frame_cards.append((frame_card, cnt))
        #frame_cards.append((warped_frame_card, cnt))

        frame_cards.append((warped_frame_card, cnt, name_img, suit_img))

    return frame_cards


In [1006]:
def find_matches(frame_card, database_card):
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(frame_card, None)
    kp2, des2 = orb.detectAndCompute(database_card, None)

    
    #bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    bf = cv2.BFMatcher()
    #matches = bf.match(des1, des2)
    matches = bf.knnMatch(des1, des2, k=2)

    '''
    index_params = dict(algorithm=6,
                        table_number=6,
                        key_size=12,
                        multi_probe_level=2)
    search_params = {}
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    '''
    
    # As per Lowe's ratio test to filter good matches
    good_matches = []
    for match in matches:
        if len(match) == 2:
            m, n = match
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

    match_img = cv2.drawMatches(frame_card, kp1, database_card, kp2, good_matches, None)

    #if len(good_matches) > 50:
        #src_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        #dst_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        #m, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
        #corrected_img = cv2.warpPerspective(img1, m, (img2.shape[1], img2.shape[0]))

    return good_matches, match_img


In [1007]:
def get_images_similarity_ratio(img1, img2):
    img1_h, img1_w = img1.shape
    img2_h, img2_w = img2.shape
    total_pixels_img1 = img1_h * img1_w
    total_pixels_img2 = img2_h * img2_w
    total_pixels = max(total_pixels_img1, total_pixels_img2)

    subtracted = cv2.subtract(img1, img2)
    subtracted = cv2.threshold(subtracted, 210, 255, cv2.THRESH_BINARY)[1]

    number_black_pixels = get_number_black_pixels(subtracted)

    return number_black_pixels / total_pixels

In [1008]:
cards = read_all_cards()
names = read_all_names()
suits = read_all_suits()

frame = cv2.imread(frame_test_path)
#binary_frame = create_binary(frame)
#cnts = find_contourns(binary_frame)
#cv2.drawContours(frame, cnts, -1, (0,255,0), 3)
frame_cards = extract_frame_cards(frame)
#match_imgs = []

for frame_card_img, cnt, frame_name_img, frame_suit_img in frame_cards:
    name_candidate = None
    max_name_similarity = -1
    suit_candidate = None
    max_suit_similarity = -1

    # Compare frame card name image with database name images
    for name_filename in names:
        name_img = names[name_filename]
        name_img_h, name_img_w = name_img.shape
        frame_name_img = cv2.resize(frame_name_img, (name_img_w, name_img_h))
        
        similarity = get_images_similarity_ratio(frame_name_img, name_img)
        if similarity > max_name_similarity:
            max_name_similarity = similarity
            name_candidate = extract_card_name_or_suit(name_filename)

    # Compare frame card suit image with database suit images
    for suit_filename in suits:
        suit_img = suits[suit_filename]
        suit_img_h, suit_img_w = suit_img.shape
        frame_suit_img = cv2.resize(frame_suit_img, (suit_img_w, suit_img_h))
        #cv2.imshow(f"{suit_filename} Frame Suit", frame_suit_img)
        #cv2.imshow(f"{suit_filename} Suit", suit_img)
        #subtracted = cv2.subtract(frame_suit_img, suit_img)
        #cv2.imshow(f"{suit_filename} Subtracted", subtracted)

        similarity = get_images_similarity_ratio(frame_suit_img, suit_img)
        if similarity > max_suit_similarity:
            max_suit_similarity = similarity
            suit_candidate = extract_card_name_or_suit(suit_filename)

    card_final_name = name_candidate + suit_candidate



    #l = []
    #max_matches = -1
    #match_img_to_show = None
    #best_database_card = None
    #for database_card in database_cards:
    #    matches, match_img = find_matches(frame_card, database_card.get_img())
    #    if len(matches) > max_matches:
    #        match_img_to_show = match_img
    #        best_database_card = database_card
    #        max_matches = len(matches)
    #    l.append(len(matches))
    
    #match_imgs.append(match_img_to_show)
    #print(max_matches)
    #print(l)
    #cv2.drawContours(frame_card_img,[cnt],-1,(255,0,0),3)
    #x,y,w,h = cv2.boundingRect(cnt)
    #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    #cv2.putText(frame, name_candidate + " " + suit_candidate,(x+10,y+30), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2)
    print(name_candidate + " " + suit_candidate)


2 S
A C
K D
A H


In [1009]:
cv2.imshow("Frame", frame)
#for i in range(len(match_imgs)):
#    cv2.imshow(f"Match Image {i+1}", match_imgs[i])
cv2.waitKey(0)
cv2.destroyAllWindows()

13